Fonction qui retourne quelles colonnes vont faire partie du master dans un noeud

In [ ]:
function calculate_columns(noeud)
    noeud_col=Array{Array{Int,1},1}() # Ensemble des colonnes d'un noeud
    
    # On ajoute une colonne artificielle au cas où
    push!(noeud_col,vcat(1,ones(Int,ne(pool)+1)))
    for c in 2:size(column_pool,1)
        add = true
        # On ajoute pas une colonne dans le noeud si :
        # 1. On a la contrainte de branchement x[i]<=0 et l'arc i est dans la colonne
        for arc in tree[noeud].setzero
            if column_pool[c][arc+1] == 1
                add = false
            end
        end
        # 2. On a la contrainte de branchement x[i]>=1 et l'arc i n'est pas dans la colonne
        for arc in tree[noeud].setone
            if column_pool[c][arc+1] == 0
                add = false
            end
        end
        
        if add push!(noeud_col,column_pool[c]) end 
    end
    
    return noeud_col
        
end

In [1]:
# Transforme la solution α en une solution x
# où les indices de x correspondent aux arcs
function calculate_xsol(mastersol,noeud_col)
    x = Array{Float32,1}(undef,ne(pool))
    for edge in 1:ne(pool)
        x[edge] = sum(noeud_col[c][edge+1]*mastersol[c] for c in 1:size(mastersol,1))
    end
    #print("à quoi ressemble la solution ? ",x,"\n")
    return x
end

calculate_xsol (generic function with 1 method)